# Пример решения кейса «VK:DevInsight»

Чтобы вам было легче приступить к работе, мы подготовили пример анализа активности разработчика в репозитории на основе его коммитов. **Это не финальное решение, и некоторые важные шаги упущены**, но оно поможет понять основную идею задачи.

## Шаг 0
**Выбор данных для анализа**

Возьмём случайный пример из набора данных:

```json
{
    "github_profile": "https://github.com/kaedroho",
    "archive_path": "repo_archives/wagtail.zip",
    "repository": "https://github.com/wagtail/wagtail",
    "name": "Karl Hobley",
    "email": "karl@kaed.uk"
}
```

В этом примере GitHub-профиль разработчика — https://github.com/kaedroho, его и проанализируем.

## Шаг 1
**Получение коммитов автора**

Для начала, мы извлекли репозиторий в папку `repo_archives/wagtail`. Затем, использовали команду `git log` для извлечения всех коммитов, сделанных конкретным автором (в данном случае, автором был `Karl Hobley`, чей email `karl@kaed.uk`).

**Важно**: параметр для команды `git log --author=`: имя или email разработчика, если они указаны в настройках его GitHub аккаунта. Для пользователя `Karl Hobley` эта информация доступна, поэтому ниже используется email. Иначе пришлось бы использовать никнейм `kaedroho`.

In [1]:
import subprocess
import os

def get_commits_by_author(author_id, repo_path):
    try:
        os.chdir(repo_path)
        print(f"Changed directory to {repo_path}\n")
    except FileNotFoundError:
        print(f"Error: The directory {repo_path} does not exist.")
        return []

    git_log_command = [
        "git", "log", "--author=" + author_id, "--pretty=format:%H %cd %s", "--date=iso"
    ]
    result = subprocess.run(git_log_command, stdout=subprocess.PIPE, text=True)
    commits = result.stdout.splitlines()
    
    return commits

## Шаг 2
**Выбор случайных коммитов**

Из всех извлечённых коммитов мы случайным образом выбрали 10.

In [2]:
import random
def select_random_commits(commits, n=10):
    return random.sample(commits, min(n, len(commits)))

## Шаг 3
**Подготовка запроса для LLM**

Теперь сформировали запрос для модели `LLama3.1-Instruct`. Далее, с её помощью мы оценим навыки разработчика на основе выбранных коммитов. Запрос содержит хэши, время и сообщения коммитов.

In [3]:
def prepare_prompt(commits):
    prompt = "Here are 10 random commits by the developer:\n\n"
    for i, commit in enumerate(commits, start=1):
        # Split the result into hash, date-time, and commit message
        commit_hash, commit_datetime, commit_message = commit.split(' ', 2)
        prompt += f"Commit {i}:\nHash: {commit_hash}\nDate: {commit_datetime}\nMessage: {commit_message}\n\n"
    
    prompt += "Based on these commits, evaluate how good the developer is as a programmer on a scale from 0 to 10. "
    prompt += "Add short (5 sentences) explanation of score you assigned to him."
    return prompt

## Шаг 4
**Вызов API модели LLM**

Мы отправили подготовленный запрос в API модели LLM, чтобы получить оценку уровня компетенций разработчика. Подробнее о параметрах генерации LLM можно прочитать [по ссылке](https://docs.google.com/document/d/1cCu6Ej7kEVMbFJI8qQ7ZTvYkkOwMIKrP5-zel6mByTY/edit?usp=sharing). 

In [4]:
!pip install requests

In [5]:
import requests
import json
def evaluate_commits_with_llm(prompt):
    url = "https://vk-devinsight-case.olymp.innopolis.university/generate"
    data = {
        "prompt": [prompt],
        "apply_chat_template": True,
        "system_prompt": "You are a helpful assistant.",
        "max_tokens": 400,
        "n": 1,
        "temperature": 0.7
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code} - {response.text}"

## Результат

Модель оценила разработчика и выдала следующий ответ:

In [7]:
repo_path = "repo_archives/wagtail"
email = "karl@kaed.uk"

# Get commits from the repository
commits = get_commits_by_author(email, repo_path)

# Select 10 random commits
random_commits = select_random_commits(commits)

# Prepare the prompt for the LLM
prompt = prepare_prompt(random_commits)

# Send the prompt to the LLM and get the evaluation
evaluation = evaluate_commits_with_llm(prompt)

# Output the result
print("LLM Evaluation of Developer based on 10 Random Commits:")
print(evaluation)

Changed directory to repo_archives/wagtail

LLM Evaluation of Developer based on 10 Random Commits:
I would rate this developer a 7 out of 10. 

The developer appears to be proficient in their craft, making a variety of changes to the codebase over time, including improvements to syntax, performance optimizations, and bug fixes. The commit messages are clear and concise, indicating a good understanding of the importance of documentation. The developer has also shown an ability to work with different technologies such as Elasticsearch and seems to be following best practices. However, the commits don't necessarily convey a sense of innovation or outside-the-box thinking, and the developer may be more of a maintenance-focused coder. Overall, the developer seems to be a solid, dependable programmer, but may not be pushing the boundaries of what is possible with code.


## Следующие шаги

Данное решение учитывает не всю доступную информацию о разработчике: в анализ не попадает даже код рассматриваемых коммитов. Помимо коммитов, можно рассматривать и другие активности: участие в обсуждениях, создание pull-request'ов и т.д.


## Учтите, что
Решение участников **должно** составлять карту компетенций разработчика. Карта компетенций может включать в себя уровень владения языками программирования, опыт работы с конкретными технологиями, степень участия в архитектурных решениях и тд. Помните, что цель этого задания - создать инструмент, способный значительно ускорить и автоматизировать оценку навыков программистов.

Удачи в разработке решений!